In [ ]:
!pip
install - q
diffusers
transformers
accelerate
deepface
pandas

import torch
from diffusers import StableDiffusionPipeline
import os
import pandas as pd
from deepface import DeepFace
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using Device: {device}")


model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.safety_checker = None  # Disable safety checker to speed up and avoid false positives

ROLES = ["doctor", "nurse", "teacher", "scientist", "chef", "artist", "builder", "driver"]
COUNT_PER_ROLE = 50
OUTPUT_DIR = "ai_images"
CSV_NAME = "analysis_ai.csv"

print(f"\n Starting Generation: {COUNT_PER_ROLE * len(ROLES)} images total...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

for role in ROLES:
    role_dir = os.path.join(OUTPUT_DIR, role)
    os.makedirs(role_dir, exist_ok=True)

    for i in tqdm(range(COUNT_PER_ROLE), desc=f"Generating {role.upper()}"):
        fname = f"{role}_{i:03d}.png"
        path = os.path.join(role_dir, fname)

        prompt = f"a photo of a {role}, looking at camera, realistic, 8k"
        negative_prompt = "cartoon, drawing, anime, illustration, ugly, deformed"

        image = pipe(prompt, negative_prompt=negative_prompt, width=512, height=512, num_inference_steps=25).images[0]
        image.save(path)

print("\nGeneration Complete!")

# 5. Analyze with DeepFace
print("\nStarting Analysis...")
results = []

for role in ROLES:
    role_dir = os.path.join(OUTPUT_DIR, role)
    images = [f for f in os.listdir(role_dir) if f.endswith(".png")]

    for img in tqdm(images, desc=f"Analyzing {role}"):
        full_path = os.path.join(role_dir, img)
        try:
            # Run DeepFace
            obj = DeepFace.analyze(
                img_path=full_path,
                actions=['gender', 'race'],
                detector_backend='opencv',
                silent=True,
                enforce_detection=False
            )

            res = obj[0]
            gender = max(res['gender'], key=res['gender'].get)
            race = res['dominant_race']
            face_detected = True
        except:
            gender = "unknown"
            race = "unknown"
            face_detected = False

        results.append({
            "image_id": img,
            "role": role,
            "face_detected": face_detected,
            "gender": gender,
            "race": race
        })

df = pd.DataFrame(results)
df.to_csv(CSV_NAME, index=False)
print(f"\nDONE! Saved to {CSV_NAME}")